# Ant Colony Optimization (ACO)

Is a probabilistic technique used in CS and operations research to solve computational problems that involve finding optimal paths through graphs, inspired by the natural behavior of ants finding the shortest path between their colony and a food source. 

All the nodes are connected to each other, therefore it's a complete graph. The best option to store this is to keep the upper or lower triangle, because the matrix is symmetric.

### How it works

In simple words, this is how it works: some ants randomly explore the graph and return to the same place where they started, leaving pheromones on the path they followed depending on the quality (how short) of the path. Then, the next ants (depending on the parameters) follow the pheromones but also tend to follow the edge with low cost.

Over time, the pheromone trails evaporate. This is crucial because it prevents the algorithm from getting stuck in a bad solution (local optimum). The pheromone on shorter paths is reinforced faster than it evaporates. Eventually, the majority of the ants converge on the single most efficient path.

### Aplications

- TSP.

- Network routing: optimizing data paths in telecommunications.

- Scheduling: Allocating resources in factories or cloud computing.

In [43]:
import random as rand

Formula to access an index: 

$$Index = i \times \text{cities} - \frac{i(i+1)}{2} + (j - i - 1)$$

In [44]:
distances = [6, 9, 17, 13, 21, 19, 21, 12, 18, 20, 23, 11, 15, 10, 21]
visibility = [1/x for x in distances]
pheromones = [0.1]*len(distances)
tabu = {
    0: [0],
    1: [1],
    2: [2],
    3: [3],
    4: [4],
    5: [5]
}

In [45]:
# Main parameters 

p = 0.2 # How much the pheromone trails evaporate
Q = 1 # A constant, usually 1
a = 1.5 # Relative importance of the pheromones 
b = 0.8 # Relative importance of the edge's visibility
ants = 6
cities = 6
iterations = 50

In [46]:
def obtain_index(i, j):

    if i>j:
        aux = j
        j = i
        i = aux
    
    return i * cities - (i*(i+1)//2) + (j - i - 1)

In [47]:
def path(origin, destiny):
        
    index = int(obtain_index(origin, destiny))
    # Get the values from the indexes
    pheromone = pheromones[index]
    vis = visibility[index]
    
    return pheromone**a * vis**b


In [48]:
def roullete(probabilities):
    
    sum_p = sum(x[1] for x in probabilities)
    p = [p[1]/sum_p for p in probabilities]
    acum = 0
    random = rand.uniform(0,1)

    for i, prob in enumerate(p):
        acum += prob
        if acum > random:
            return probabilities[i][0]
    

In [49]:
def draw_path(path, ant):
    path1 = path.copy()
    path1.pop()
    print(f"La hormiga {ant} siguió el siguiente camino:")
    for node in path1:
        print(f"{node}->", end="")
    print(ant)

In [50]:
def total_weight(path):
    sum = 0
    
    for i, _ in enumerate(path):
        
        if i == len(path) - 1:
            print(f'Distancia total: {sum}')
            return Q/sum
        
        index = int(obtain_index(path[i], path[i+1]))
        distance = distances[index]
        sum += distance
        

In [51]:
def match(all_paths, i, j, weights):
    
    sum = 0
    
    for index, node in enumerate(all_paths):
        for x, _ in enumerate(node):
            
            if x == len(node)-1:
                break
            
            if (node[x]==i and node[x+1]==j) or (node[x]==j and node[x+1]==i):
                sum += weights[index]
    
    return sum

In [52]:
def update_pheromones(weights, all_paths, pheromones):
    
    new_pheromones = []
    sum = 0
    
    for i in range(cities):
        for j in range(i+1, cities):
            sum = match(all_paths, i, j, weights)
            result = (1-p) * pheromones[int(obtain_index(i, j))]+ sum
            new_pheromones.append(result)
    
    return new_pheromones

In [53]:

for i in range(iterations):
    
    print(f"ITERATION {i}\n")
    
    weights = []
    tabu = {
    0: [0],
    1: [1],
    2: [2],
    3: [3],
    4: [4],
    5: [5]
    }
        
    for ant in range(ants):
        next_node = ant
        while len(tabu[ant]) < cities:
            
            probabilities = []
            
            for node in range(cities):
                if node not in tabu[ant]:
                    probabilities.append([(next_node, node), path(next_node, node)])
                
                elif len(tabu[ant]) == cities-1 and node not in tabu[ant]:
                    tabu[ant].append(node)
                    break
            
            if len(tabu[ant]) < cities:
                path_to_follow = roullete(probabilities)
                next_node = path_to_follow[1]
                tabu[ant].append(next_node)
            
        tabu[ant].append(ant)
        draw_path(tabu[ant], ant)
    
    print()
    
    # PESO TOTAL POR RECORRIDO
    for ant in range(ants):
        weights.append(total_weight(tabu[ant]))
    
    all_paths = [x for x in (tabu[y] for y in range(ants))]
        
    pheromones = update_pheromones(weights, all_paths, pheromones)
    print()
        

ITERATION 0

La hormiga 0 siguió el siguiente camino:
0->4->3->5->2->1->0
La hormiga 1 siguió el siguiente camino:
1->0->3->4->5->2->1
La hormiga 2 siguió el siguiente camino:
2->0->5->3->4->1->2
La hormiga 3 siguió el siguiente camino:
3->0->1->5->4->2->3
La hormiga 4 siguió el siguiente camino:
4->5->2->0->3->1->4
La hormiga 5 siguió el siguiente camino:
5->2->0->1->3->4->5

Distancia total: 74
Distancia total: 89
Distancia total: 86
Distancia total: 105
Distancia total: 91
Distancia total: 83

ITERATION 1

La hormiga 0 siguió el siguiente camino:
0->2->5->3->1->4->0
La hormiga 1 siguió el siguiente camino:
1->0->2->4->5->3->1
La hormiga 2 siguió el siguiente camino:
2->1->3->4->5->0->2
La hormiga 3 siguió el siguiente camino:
3->5->2->0->1->4->3
La hormiga 4 siguió el siguiente camino:
4->3->5->1->0->2->4
La hormiga 5 siguió el siguiente camino:
5->2->0->1->3->4->5

Distancia total: 76
Distancia total: 90
Distancia total: 106
Distancia total: 63
Distancia total: 81
Distancia total: 

In [54]:
import itertools

def brute_force_tsp(distances, cities):
    best = None
    best_cost = float("inf")
    
    for perm in itertools.permutations(range(cities)):
        cost = 0
        valid = True
        
        for i in range(cities - 1):
            a, b = perm[i], perm[i+1]
            index = obtain_index(a, b)
            cost += distances[index]
        
        # regresar al origen
        index = obtain_index(perm[-1], perm[0])
        cost += distances[index]
        
        if cost < best_cost:
            best_cost = cost
            best = perm
    
    return best, best_cost


In [55]:
brute_force_tsp(distances, 6)

((0, 1, 4, 3, 5, 2), 63)